In [103]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OrdinalEncoder

In [104]:
X = datasets.fetch_covtype().data[:5000]
y = datasets.fetch_covtype().target[:5000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(X_train.shape, y_train.shape)

enc = OrdinalEncoder()
y_train = enc.fit_transform(y_train.reshape(-1, 1)).reshape(-1, )
y_test = enc.transform(y_test.reshape(-1, 1)).reshape(-1, )
print(np.unique(y_train))  # 7分类任务

X_train, X_test, y_train, y_test = tf.constant(X_train), tf.constant(X_test), tf.constant(y_train), tf.constant(y_test)

(4500, 54) (4500,)
[0. 1. 2. 3. 4. 5. 6.]


In [105]:
# Model
def get_model():
    inputs = tf.keras.Input(shape=(54), name="dcdmm")
    x = tf.keras.layers.Dense(256, activation='tanh')(inputs)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(128)(x)
    predictions = tf.keras.layers.Dense(7)(x)
    model = tf.keras.Model(inputs=inputs, outputs=predictions)
    model.summary()
    return model

In [106]:
model = get_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dcdmm (InputLayer)          [(None, 54)]              0         
                                                                 
 dense_64 (Dense)            (None, 256)               14080     
                                                                 
 dropout_16 (Dropout)        (None, 256)               0         
                                                                 
 dense_65 (Dense)            (None, 128)               32896     
                                                                 
 dense_66 (Dense)            (None, 128)               16512     
                                                                 
 dense_67 (Dense)            (None, 7)                 903       
                                                                 
Total params: 64,391
Trainable params: 64,391
Non-trainabl

In [107]:
predictions = model.predict(X_test[:10],  # 预测10条样本
                            batch_size=32, verbose=0)
print('predictions shape:', predictions)

predictions shape: [[-1.7491801   0.92415035  1.6891038   0.8388096   1.954925    1.797978
  -4.291072  ]
 [-1.97315    -0.22237208  0.35297367  3.5076663   1.2377771   0.6117433
  -4.3779664 ]
 [-2.3347988  -0.5105111   1.4720365   2.6268973   2.1093996   0.89499533
  -4.03901   ]
 [-1.4302342   1.371626    1.4153801  -0.4038673   2.2131233   1.9235908
  -3.9911106 ]
 [ 3.33393     4.3550963  -3.8964312  -1.804666    3.1911123  -3.7489622
  -2.3664224 ]
 [ 2.978585    2.3242633  -1.3965076  -2.7500482   0.2667422  -1.9311521
   0.04668408]
 [ 3.3544025   4.2420897  -3.0444758  -4.284931    1.3081927  -2.8169656
   1.3390887 ]
 [ 2.7080727   2.9850156  -2.985154   -3.857289    0.57757103 -2.8397806
   2.2626834 ]
 [ 1.4015707   2.00615    -2.3947647  -1.4882579   0.07199524 -1.667265
  -0.0232187 ]
 [ 1.2841535   1.2613521  -3.0828567  -3.4780428   0.8023448  -1.9987361
   3.0943058 ]]


### 方法一:save_weights

In [108]:
# Saves all layer weights.
model.save_weights('save_weights.h5')

In [109]:
model_k_h5 = get_model()  # 重新创建网络
model_k_h5.load_weights('save_weights.h5')
model_k_h5.predict(X_test[:10], batch_size=32, verbose=0)

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dcdmm (InputLayer)          [(None, 54)]              0         
                                                                 
 dense_68 (Dense)            (None, 256)               14080     
                                                                 
 dropout_17 (Dropout)        (None, 256)               0         
                                                                 
 dense_69 (Dense)            (None, 128)               32896     
                                                                 
 dense_70 (Dense)            (None, 128)               16512     
                                                                 
 dense_71 (Dense)            (None, 7)                 903       
                                                                 
Total params: 64,391
Trainable params: 64,391
Non-trainabl

array([[-1.7491801 ,  0.92415035,  1.6891038 ,  0.8388096 ,  1.954925  ,
         1.797978  , -4.291072  ],
       [-1.97315   , -0.22237208,  0.35297367,  3.5076663 ,  1.2377771 ,
         0.6117433 , -4.3779664 ],
       [-2.3347988 , -0.5105111 ,  1.4720365 ,  2.6268973 ,  2.1093996 ,
         0.89499533, -4.03901   ],
       [-1.4302342 ,  1.371626  ,  1.4153801 , -0.4038673 ,  2.2131233 ,
         1.9235908 , -3.9911106 ],
       [ 3.33393   ,  4.3550963 , -3.8964312 , -1.804666  ,  3.1911123 ,
        -3.7489622 , -2.3664224 ],
       [ 2.978585  ,  2.3242633 , -1.3965076 , -2.7500482 ,  0.2667422 ,
        -1.9311521 ,  0.04668408],
       [ 3.3544025 ,  4.2420897 , -3.0444758 , -4.284931  ,  1.3081927 ,
        -2.8169656 ,  1.3390887 ],
       [ 2.7080727 ,  2.9850156 , -2.985154  , -3.857289  ,  0.57757103,
        -2.8397806 ,  2.2626834 ],
       [ 1.4015707 ,  2.00615   , -2.3947647 , -1.4882579 ,  0.07199524,
        -1.667265  , -0.0232187 ],
       [ 1.2841535 ,  1.2613

### 方法二:save

In [110]:
# Saves the model to Tensorflow SavedModel or a single HDF5 file.
model.save("save.h5")  # HDF5 file

# model.save('keras_model_tf_version', save_format='tf')  # 也可以保存为Tensorflow SavedModel

In [111]:
new_model = tf.keras.models.load_model('save.h5')  # 不需要重新创建网络
new_model.predict(X_test[:10], batch_size=32, verbose=0)

array([[-1.7491801 ,  0.92415035,  1.6891038 ,  0.8388096 ,  1.954925  ,
         1.797978  , -4.291072  ],
       [-1.97315   , -0.22237208,  0.35297367,  3.5076663 ,  1.2377771 ,
         0.6117433 , -4.3779664 ],
       [-2.3347988 , -0.5105111 ,  1.4720365 ,  2.6268973 ,  2.1093996 ,
         0.89499533, -4.03901   ],
       [-1.4302342 ,  1.371626  ,  1.4153801 , -0.4038673 ,  2.2131233 ,
         1.9235908 , -3.9911106 ],
       [ 3.33393   ,  4.3550963 , -3.8964312 , -1.804666  ,  3.1911123 ,
        -3.7489622 , -2.3664224 ],
       [ 2.978585  ,  2.3242633 , -1.3965076 , -2.7500482 ,  0.2667422 ,
        -1.9311521 ,  0.04668408],
       [ 3.3544025 ,  4.2420897 , -3.0444758 , -4.284931  ,  1.3081927 ,
        -2.8169656 ,  1.3390887 ],
       [ 2.7080727 ,  2.9850156 , -2.985154  , -3.857289  ,  0.57757103,
        -2.8397806 ,  2.2626834 ],
       [ 1.4015707 ,  2.00615   , -2.3947647 , -1.4882579 ,  0.07199524,
        -1.667265  , -0.0232187 ],
       [ 1.2841535 ,  1.2613

### 方法三:tf.savaed_model.save

In [112]:
# 一般用于模型部署
tf.saved_model.save(model, 'tf_saved_model_save')

INFO:tensorflow:Assets written to: tf_saved_model_save\assets


In [113]:
!saved_model_cli show --dir tf_saved_model_save --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dcdmm'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 54)
        name: serving_default_dcdmm:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_67'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 7)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          inputs: TensorSpec(shape=(Non

In [114]:
'''
signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dcdmm'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 54)
        name: serving_default_dcdmm:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_27'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 7)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
'''
restored_saved_model = tf.saved_model.load('tf_saved_model_save')
f = restored_saved_model.signatures["serving_default"]
f(dcdmm=tf.cast(X_test[:10], tf.float32))

{'dense_67': <tf.Tensor: shape=(10, 7), dtype=float32, numpy=
 array([[-1.7491801 ,  0.92415035,  1.6891038 ,  0.8388096 ,  1.954925  ,
          1.797978  , -4.291072  ],
        [-1.97315   , -0.22237208,  0.35297367,  3.5076663 ,  1.2377771 ,
          0.6117433 , -4.3779664 ],
        [-2.3347988 , -0.5105111 ,  1.4720365 ,  2.6268973 ,  2.1093996 ,
          0.89499533, -4.03901   ],
        [-1.4302342 ,  1.371626  ,  1.4153801 , -0.4038673 ,  2.2131233 ,
          1.9235908 , -3.9911106 ],
        [ 3.33393   ,  4.3550963 , -3.8964312 , -1.804666  ,  3.1911123 ,
         -3.7489622 , -2.3664224 ],
        [ 2.978585  ,  2.3242633 , -1.3965076 , -2.7500482 ,  0.2667422 ,
         -1.9311521 ,  0.04668408],
        [ 3.3544025 ,  4.2420897 , -3.0444758 , -4.284931  ,  1.3081927 ,
         -2.8169656 ,  1.3390887 ],
        [ 2.7080727 ,  2.9850156 , -2.985154  , -3.857289  ,  0.57757103,
         -2.8397806 ,  2.2626834 ],
        [ 1.4015707 ,  2.00615   , -2.3947647 , -1.4882579